# Implementación del algoritmo Apriori

##### Por: Daniela Flores Villanueva

En este *notebook* se define lo necesario para implementar el algoritmo Apriori con la menor dependencia de código externo posible. Posteriormente, se emplea la implementación propuesta para minar reglas de asociación que presenten altos valores para ciertas métricas y se analizan separadamente. Finalmente, se entrega una forma de visualizar un conjunto de reglas de asociación.

## Librerías utilizadas

- `numpy`: se utilizó para cargar los datos del Spotify RecSys Challenge, que venían en una arreglo serializado de `numpy`.
- `collections`: se usó su clase `defaultdict`, para formar los diccionarios con los contadores de soporte de cada *itemset*.
- `itertools`: se empleó su método `combinations`, para generar todos los subconjuntos de largo `k` que pudiesen extraerse de un conjunto.

In [1]:
import numpy as np
import collections
import pandas as pd

In [2]:
def combinations(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i + 1, r):
            indices[j] = indices[j - 1] + 1
        yield tuple(pool[i] for i in indices)

In [3]:
list(combinations([1,2,3,4], 3))

[(1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4)]

## Clase Apriori

La implementación presentada en este *notebook* se basa en la creación de un objeto de clase `Apriori` que, en primer lugar, recibe los siguientes parámetros:

- `data`: corresponde a las transacciones, en forma de `numpy.array`.
- `min_support`: soporte mínimo para que un `itemset` sea considerado frecuente.
- `min_confidence`: confianza mínima para que cierta regla de asociación sea retornada.
- `min_lift`: *lift* mínimo para que cierta regla de asociación sea retornada.

### Métodos

- `prepare_data`: convierte los datos recibidos (`self.data`) en una lista de conjuntos, donde cada conjunto corresponde a una lista de reproducción. De esta forma, se evitan canciones repetidas en las listas. También, se guarda en `self.songs_counter` un contador con la frecuencia de cada canción. Esta información permitirá formar el conjunto L_1.
- `get_songs_appearances`: gracias a este método, se guarda un diccionario cuyas llaves son nombres de canciones y los valores el conjunto de índices de las listas de reproducción donde aparece dicha canción. Esto permitirá obtener el contador de soporte de un *itemset* con facilidad.
- `generate_L_1`: con esto se obtienen los *itemsets* frecuentes de largo 1, es decir, las canciones que aparecen un porcentaje de veces mayor al mínimo soporte definido.
- `has_unfrequent_subset`: este método permite verificar la propiedad de monotonicidad de un *k-itemset*: si el itemset es frecuente, entonces todos sus subconjuntos deben serlo también. Este método finalmente no se ocupó, pues demoraba la ejecución del método `fit`. Cabe destacar, eso sí, que la poda (realizada en `prune_itemsets`) es lo suficientemente veloz como para prescindir de este paso al menos en esta implementación. Lo correcto hubiese sido, por supuesto, incluir este paso de verificación al generar candidatos a *itemsets* frecuentes.
- `generate_new_candidates`: este método permite generar un nuevo conjunto de posibles *itemsets* frecuentes. Esto se hace a partir de la unión de un conjunto de *itemsets* frecuentes consigo mismo. Un *itemset* se "fusiona" con otro solo si sus primeros $k - 2$ elementos son iguales. Puede observarse que se dejó comentada la verificación de la propiedad de monotonocidad (`if not self.has_unfrequent_subset(c, current_itemsets, k)`), puesto que su utilización, aunque correcta, agrega alrededor de 8 segundos extra al tiempo de ejecución del método `fit`.
- `calculate_subset_count`: calcula el contador de soporte de un *itemset*. Gracias al diccionario generado con `get_songs_appearances`, es fácil obtener el contador de soporte de un *itemset*: basta con obtener las listas de reproducción en que aparece cada canción del *itemset* y luego intersectar esos conjuntos de *playlists*. El largo de la intersección corresponderá al contador de soporte de *itemset*.
- `prune_itemsets`: retorna el conjunto de *itemsets* que superan el mínimo soporte definido.
- `fit`: método que genera los *itemsets* frecuentes.
- `get_association_rule_from_itemset`: Dado un *itemset* $I$ se forman todas las combinaciones posibles de $X \rightarrow Y$, donde $X \subset I$ y $Y = I - X$. Es importante notar que solo se consideran reglas de asociación aquellas en que tanto X como Y tengan largo mayor o igual a 1.
- `generate`: se generan todas las reglas de asociación posibles a partir de los *itemsets* frecuentes encontrados. Por defecto, se retornan ordenadas por *confidence*. El usuario puede cambiar esto mediante la modificación del parámetro `order_by`.
- `get_n_top_rules`: permite obtener las $n$ reglas con mayor valor de la métrica determinada por el usuario (*confidence*, por defecto). Esta métrica puede corresponder a *confidence* o a *lift*. El criterio de ordenación se cambia a través de la modificación del parámetro `order_by`.

In [4]:
class Apriori:
    def __init__(self, data, min_support=0.01, min_confidence=0.01,
                 min_lift=1):
        self.data = data
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.min_lift = min_lift

    def prepare_data(self):
        self.playlists = list(self.data.item().values())
        self.playlists = [set(playlist) for playlist in self.playlists]
        unique_songs = [item for sublist in self.playlists for item in sublist]
        self.songs_counter = pd.Series(
            data=unique_songs).value_counts().to_dict()

    def get_songs_appearances(self):
        songs_in_playlists = collections.defaultdict(set)
        for index, playlist in enumerate(self.playlists):
            for song in playlist:
                songs_in_playlists[song].add(index)
        self.songs_in_playlists = songs_in_playlists

    def generate_L_1(self):
        self.L_1_counter = {
            song: times
            for song, times in self.songs_counter.items()
            if times / len(self.playlists) >= self.min_support
        }
        self.L_1 = [{song} for song in self.L_1_counter.keys()]

    def has_unfrequent_subset(self, candidate, current_itemsets, k):
        subsets = combinations(candidate, k - 1)
        for subset in subsets:
            if frozenset(subset) not in current_itemsets:
                return True
        return False

    def generate_new_candidates(self, current_itemsets, k):
        C_k = set()
        m = k - 2
        for candidate in current_itemsets:
            candidate = list(candidate)
            for aux_candidate in current_itemsets:
                aux_candidate = list(aux_candidate)
                can_join = True
                for i in range(k - 2):
                    if candidate[i] != aux_candidate[i]:
                        can_join = False
                        break
                if not can_join:
                    continue
                if candidate[k - 2] < aux_candidate[k - 2]:
                    c = candidate + [aux_candidate[k - 2]]
                    c = frozenset(sorted(c))
                    #                     if not self.has_unfrequent_subset(c, current_itemsets, k):
                    #                         C_k.add(c)
                    C_k.add(c)
        return C_k

    def calculate_subset_count(self, subset):
        playlists_inter = []
        for song in subset:
            playlists_inter.append(self.songs_in_playlists[song])
        return len(set.intersection(*playlists_inter))

    def prune_itemsets(self, C_k):
        C_k_counter = {}
        playlist_length = len(self.playlists)
        for candidate in C_k:
            C_k_counter[candidate] = self.calculate_subset_count(candidate)
        L_k_counter = {
            subset: times
            for subset, times in C_k_counter.items()
            if times / playlist_length >= self.min_support
        }
        return L_k_counter

    def fit(self):
        self.prepare_data()
        self.get_songs_appearances()
        self.generate_L_1()
        self.k_frequent_itemsets = {}
        self.k_frequent_itemsets[1] = sorted(
            self.L_1_counter.items(), key=lambda x: x[1], reverse=True)
        self.frequent_itemsets = []
        k = 2
        current = self.L_1
        while len(current) != 0:
            C_k = self.generate_new_candidates(current, k)
            L_k_counter = self.prune_itemsets(C_k)
            L_k = L_k_counter.keys()
            self.frequent_itemsets.extend(L_k)
            self.k_frequent_itemsets[k] = sorted(
                L_k_counter.items(), key=lambda x: x[1], reverse=True)
            k += 1
            current = L_k
        print(self.k_frequent_itemsets)

    def get_association_rule_from_itemset(self, itemset):
        itemset_count = self.calculate_subset_count(itemset)
        itemset_support = itemset_count / len(self.playlists)
        for i in range(1, len(itemset) + 1):
            for x_set in itertools.combinations(itemset, i):
                x_set = frozenset(x_set)
                y_set = frozenset(itemset) - x_set
                x_support = self.calculate_subset_count(x_set) / len(
                    self.playlists)
                x_y_support = self.calculate_subset_count(
                    x_set.union(y_set)) / len(self.playlists)
                rule_confidence = x_y_support / x_support
                if len(x_set) > 0 and len(y_set) > 0:
                    y_support = self.calculate_subset_count(y_set) / len(
                        self.playlists)
                    rule_lift = x_y_support / (x_support * y_support)
                    self.rules["{} -> {}".format(x_set, y_set)] = {
                        "confidence": rule_confidence,
                        "lift": rule_lift
                    }

    def generate(self, order_by="confidence"):
        self.rules = {}
        for itemset in self.frequent_itemsets:
            self.get_association_rule_from_itemset(itemset)
        sorted_rules = sorted(
            self.rules.items(), key=lambda x: x[1][order_by], reverse=True)
        for rule, metrics in sorted_rules:
            x, y = rule.split("->")
            print("{}->{}\nConfidence: {}\nLift: {}\n".format(
                x, y, metrics["confidence"], metrics["lift"]))

    def get_n_top_rules(self, n, order_by="confidence"):
        sorted_rules = sorted(
            self.rules.items(), key=lambda x: x[1][order_by],
            reverse=True)[0:n]
        for rule, metrics in sorted_rules:
            x, y = rule.split("->")
            print("{}->{}\nConfidence: {}\nLift: {}\n".format(
                x, y, metrics["confidence"], metrics["lift"]))

In [5]:
spotify_data = np.load("spotify.npy")

In [6]:
apriori = Apriori(
    data=spotify_data, min_support=0.01, min_confidence=0.03, min_lift=0.9)

In [7]:
apriori.fit()

{1: [('Closer', 723), ('HUMBLE.', 465), ('Home', 454), ('Roses', 424), ('Ride', 414), ('One Dance', 411), ('Congratulations', 403), ('Broccoli (feat. Lil Yachty)', 399), ('Caroline', 390), ('Let Me Love You', 382), ('Gold', 369), ('T-Shirt', 351), ('Bad and Boujee (feat. Lil Uzi Vert)', 345), ('Stay', 339), ('Riptide', 332), ('iSpy (feat. Lil Yachty)', 331), ('Unforgettable', 326), ('Location', 323), ('XO TOUR Llif3', 323), ('No Problem (feat. Lil Wayne & 2 Chainz)', 321), ('Forever', 321), ("I'm the One", 319), ('Sorry', 318), ('Slide', 318), ("Don't Let Me Down", 317), ('Mask Off', 316), ('No Role Modelz', 311), ('Ignition - Remix', 310), ('Shape of You', 310), ('goosebumps', 308), ('Panda', 307), ("Don't", 305), ('Mercy', 305), ('White Iverson', 304), ('Fake Love', 297), ('Bounce Back', 297), ('Hello', 289), ('Trap Queen', 289), ('Jumpman', 287), ('Down', 287), ('My Girl', 286), ('Black Beatles', 285), ('Gold Digger', 285), ('Hurricane', 284), ('Despacito - Remix', 282), ('Starboy',

In [8]:
apriori.generate()

frozenset({'Mask Off', 'DNA.'}) -> frozenset({'HUMBLE.'})
Confidence: 0.9090909090909092
Lift: 19.550342130987293

frozenset({'XO TOUR Llif3', 'DNA.'}) -> frozenset({'HUMBLE.'})
Confidence: 0.864406779661017
Lift: 18.589393110989615

frozenset({'DNA.'}) -> frozenset({'HUMBLE.'})
Confidence: 0.8225108225108225
Lift: 17.68840478517898

frozenset({'Mask Off', 'XO TOUR Llif3'}) -> frozenset({'HUMBLE.'})
Confidence: 0.8036809815950922
Lift: 17.283461969786927

frozenset({'Broccoli (feat. Lil Yachty)', 'Bounce Back'}) -> frozenset({'Bad and Boujee (feat. Lil Uzi Vert)'})
Confidence: 0.7751937984496124
Lift: 22.469385462307603

frozenset({'Slippery (feat. Gucci Mane)', 'XO TOUR Llif3'}) -> frozenset({'HUMBLE.'})
Confidence: 0.7651515151515151
Lift: 16.454871293580972

frozenset({'XO TOUR Llif3', 'Tunnel Vision'}) -> frozenset({'HUMBLE.'})
Confidence: 0.7500000000000001
Lift: 16.12903225806452

frozenset({'Mask Off', 'Congratulations'}) -> frozenset({'HUMBLE.'})
Confidence: 0.7469135802469136


In [9]:
apriori.get_n_top_rules(10, "confidence")

frozenset({'Mask Off', 'DNA.'}) -> frozenset({'HUMBLE.'})
Confidence: 0.9090909090909092
Lift: 19.550342130987293

frozenset({'XO TOUR Llif3', 'DNA.'}) -> frozenset({'HUMBLE.'})
Confidence: 0.864406779661017
Lift: 18.589393110989615

frozenset({'DNA.'}) -> frozenset({'HUMBLE.'})
Confidence: 0.8225108225108225
Lift: 17.68840478517898

frozenset({'Mask Off', 'XO TOUR Llif3'}) -> frozenset({'HUMBLE.'})
Confidence: 0.8036809815950922
Lift: 17.283461969786927

frozenset({'Broccoli (feat. Lil Yachty)', 'Bounce Back'}) -> frozenset({'Bad and Boujee (feat. Lil Uzi Vert)'})
Confidence: 0.7751937984496124
Lift: 22.469385462307603

frozenset({'Slippery (feat. Gucci Mane)', 'XO TOUR Llif3'}) -> frozenset({'HUMBLE.'})
Confidence: 0.7651515151515151
Lift: 16.454871293580972

frozenset({'XO TOUR Llif3', 'Tunnel Vision'}) -> frozenset({'HUMBLE.'})
Confidence: 0.7500000000000001
Lift: 16.12903225806452

frozenset({'Mask Off', 'Congratulations'}) -> frozenset({'HUMBLE.'})
Confidence: 0.7469135802469136
